# Third party Data Import - PlanetScope, COVID 19

This Jupyter Notebook demonstrates how to import **PlanetScope** data into Sentinel Hub and then access it.
Requirements for the data import:
- AOI - few supermarkets around the Europe, given as geojson file
- all data available in April 2019 ~~with less than 50% cloud coverage~~
- all data in the period 2020-02-01 - 2020-04-02 ~~with less than 80% cloud coverage~~
    - for both periods selecet a scene every 7 days with less than or equal to 30% cloud coverage

We will use **Simple search** and **Order products**[1].

 
Useful links:  
    [1] TPDI API documentation: https://docs.sentinel-hub.com/api/latest/#/DATA_IMPORT_API/README  
    [2] TPDI API reference (Swagger docs): https://docs.sentinel-hub.com/api/latest/reference/#tag/dataimport_planet  
    [3] About PlanetScope data: https://docs.sentinel-hub.com/api/latest/#/data/PlanetScope

## Imports and credentials

Before you start an import, please:

1. fill in **config.py** file with your credentials
2. set the collection id and other setting in the cell starting with ## Settings


In [2]:
import requests 
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

import datetime
import dateutil.parser

# Import SH credentials and PLANET_API_KEY
from config import *

# Create a session
client = BackendApplicationClient(client_id=client_id)
oauth = OAuth2Session(client=client)


# Get token for the session
token = oauth.fetch_token(token_url='https://services.sentinel-hub.com/oauth/token',
                          client_id=client_id, client_secret=client_secret)

# All requests using this session will have an access token automatically added 
resp = oauth.get("https://services.sentinel-hub.com/oauth/tokeninfo")

In [ ]:
#Get my quota

url = f"https://services.sentinel-hub.com/dataimport/v1/quotas"

r = oauth.get(url=url)
r.json()

## A search function

In [41]:
def search_by_date(search_date, PLANET_API_KEY, geometry, cc):
    d1 = dateutil.parser.isoparse(search_date)
    d2 = d1 + datetime.timedelta(days=1)
    
    url = "https://services.sentinel-hub.com/dataimport/v1/search"
    
    query = {
    "provider": "PLANET",
    "planetApiKey": PLANET_API_KEY,
    "bounds": {
        "geometry": geometry
    },
    "data": [
        {
            "itemType": "PSScene4Band",
            "productBundle": "analytic",
            "dataFilter": {
                "timeRange": {
                    "from": d1.isoformat().replace('+00:00', 'Z'),
                    "to": d2.isoformat().replace('+00:00', 'Z')
                },
                "maxCloudCoverage": cc,
                "nativeFilter": {
                    "type": "AndFilter",
                    "config": [
                        {
                            "type": "PermissionFilter",
                            "config": [
                                "assets:download"
                            ]
                        },
                        {
                            "type": "AssetFilter",
                            "config": [
                                "analytic"
                            ]
                        },
                        {"type": "StringInFilter",
                        "field_name": "quality_category",
                        "config": [
                            "standard"
                            ]}
                      ]   
                    }
                }
            }
        ]
    }
    
    response = oauth.post(url, json=query)
    response.raise_for_status()
    results = response.json()
    return results
    

##  Order data import

In [42]:
## Read AOI from file

file_name = "RACE_supermarket_buffer_1.geojson" # set your file name

with open(file_name) as f: # we assigned buffer of cca 1km around given points
    data = json.load(f)

geometries = [ (feature["geometry"], feature["properties"]["CITY"]) for feature in data["features"]]

In [ ]:
## Settings
collectionId = "" # PlanetScope, Covid 19
skip_names = ["Milan"] # Set the names of geometries, which you wanna skip
area_treshold = 1000 # Set area treshhold in km^2, orders bigger than the treshold will not be confirmed

 # April 2019
#date_from = "2019-04-01T00:00:00.000Z"
#date_to = "2019-05-01T00:00:00.000Z"
#name_sufix = ""
#cc = 30

# Today 2020
date_from = "2020-02-01T00:00:00.000Z"
date_to = "2020-04-03T00:00:00.000Z"
name_sufix =", 2020"
cc =30

for geo in geometries:

    geometry = geo[0]
    name = geo[1] + name_sufix

    if name in skip_names:
        print (f"{name} skipped.")
        continue
    
    ## Search
    # Selecting a candidate date every 7 days
    candidate_dates = [dateutil.parser.isoparse(date_from)]
    inter_date = candidate_dates[0] + datetime.timedelta(days=7) 
    while temp_date < dateutil.parser.isoparse(date_to):
        candidate_dates.append(inter_date)
        inter_date = inter_date + datetime.timedelta(days=7) 
    
    # Add the date_to to candiates dates - might not be needed
    if dateutil.parser.isoparse(date_to) not in candidate_dates:
        candidate_dates.append(dateutil.parser.isoparse(date_to))

    selected_items = []

    for date in candidate_dates:
        iss = [0, 1, -1, 2, -2, 3, -3]
        for i in iss:
            print(i)
            delete = False
            date_temp = date + datetime.timedelta(days=i) 
            print(date_temp)
            results = search_by_date(date_temp.isoformat(), PLANET_API_KEY, geometry, 100)
            id_date_cc = [(feature["id"], feature['properties']["acquired"], feature['properties']["cloud_cover"]) for feature in results["features"]]
            for row in id_date_cc:
                if row[2] > cc/100:
                    delete = True
            if delete:
                id_date_cc = []
            print(id_date_cc)
            if id_date_cc:
                selected_items.extend(id_date_cc)
                print("Results found")
                break
                
    print(f"Search for {name} returns {selected_items}")
    item_ids = [item[0] for item in selected_items]
    
    if item_ids: 
        ## 2 Order products

        url = f"https://services.sentinel-hub.com/dataimport/v1/orders"
        payload = {
            "name": name,
            "collectionId": collection_id,
            "input": {
                "provider": "PLANET",
                "planetApiKey": PLANET_API_KEY,
                "bounds": {
                    "geometry": geometry
                },
                "data": [
                    {
                        "itemType": "PSScene4Band",
                        "productBundle": "analytic",
                        "itemIds": item_ids
                    }
                ]
            }
        }
        
        response = oauth.post(url, json=payload)
        response.raise_for_status()
        order = response.json()
        
        order_id = order['id']
        area = order['sqkm']
        print(f"Order {name}, id: {order_id}, area: {area} km^2")
        
        if area < area_treshold:
            ## 3 Confirm
            url = f"https://services.sentinel-hub.com/dataimport/v1/orders/{order_id}/confirm"
            response = oauth.post(url)
            response.raise_for_status()


In [10]:
# Chek Order Status
order_id = order_id

url = f"https://services.sentinel-hub.com/dataimport/v1/orders/{order_id}"

response = oauth.get(url)
response.raise_for_status()

order = response.json()
order

## List all orders

In [9]:
url = "https://services.sentinel-hub.com/dataimport/v1/orders"

response = oauth.get(url)
response.raise_for_status()

response.json()

## Get the data

To access the data using SH process API, plese refer to **Covid 19 Pleiades SUPERMARKETS** jupyter notebooks.